## Information Retrieval 24/25, University of Pisa
### Franco Maria Nardini, Rossano Venturini (francomaria.nardini@isti.cnr.it, rossano.venturini@unipi.it)

# Inverted Index

## C4 Dataset

[C4 Dataset](https://huggingface.co/datasets/allenai/c4)

A colossal, cleaned version of Common Crawl's web crawl corpus (from Google). Based on Common Crawl dataset: "https://commoncrawl.org".

We use the processed version of Google's C4 dataset by Allen Institute for AI.
They prepared five variants of the data: `en`, `en.noclean`, `en.noblocklist`, `realnewslike`, and `multilingual (mC4)`.

For reference, these are the sizes of the variants:

- `en`: 305GB
- `en.noclean`: 2.3TB
- `en.noblocklist`: 380GB
- `realnewslike`: 15GB
- `multilingual (mC4)`: 9.7TB (108 subsets, one per language)

The `en.noblocklist` variant is exactly the same as the en variant, except we turned off the so-called "badwords filter", which removes all documents that contain words from the lists at https://github.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words.

In [35]:
!pip3 install datasets nltk unidecode

  Obtaining dependency information for unidecode from https://files.pythonhosted.org/packages/84/b7/6ec57841fb67c98f52fc8e4a2d96df60059637cba077edc569a302a8ffc7/Unidecode-1.3.8-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.7 MB/s eta 0:00:00ta 0:00:01


#### We download 4 out of 1024 files of size ~318Mb compressed each

In [2]:
from datasets import load_dataset

c4_subset = load_dataset("allenai/c4", data_files="en/c4-train.0102*-of-01024.json.gz")

#### Get a list of URls and a list of corresponding documents

In [4]:
urls = [x['url'] for x in c4_subset["train"]]
documents = [x['text'] for doc_id, x in enumerate(c4_subset["train"])]

print(f"Number of documents: {len(urls)}")
print(f"Number of characters: {sum(len(x) for x in documents)}")

Number of documents: 1425269
Number of characters: 3065881920


In [4]:
c4_subset = None

In [10]:
print(urls[0])
print(documents[0])

https://americanhealthandbeauty.com/articles/2704/non-surgical-fat-reduction--zerona-vs-zeltiq
Liposuction has remained one of the most popular cosmetic surgeries for years as people turn to their doctors to remove the fat that diet and exercise can't seem to touch. Recently, there has been a trend towards less invasive aesthetic options as lasers and fillers replace facelifts and laser lipo takes center stage with traditional liposuction. There are two devices, both currently undergoing FDA testing, which could replace fat reduction surgery altogether. They are Zerona and Zeltiq, and they are pain free treatments to cut the fat.
Made by Erchonia, the Zerona laser is not exactly a new concept. Newport Beach Zerona physician Dr. Thomas Barnes has been using this low level laser therapy device in his office for several years. Zerona was originally developed to assist in the performance of traditional lipo. "I was involved with the company that developed Zerona...," says Dr. Barnes. "I've

#### Let's compute the number of distinct domains

In [6]:
def get_domain(url):
    url = url.replace("https://", "").replace("http://", "")
    domain = url.split("/")[0]
    return domain

In [7]:
domains = {}

for url in urls:
    domain = get_domain(url)
    if domain not in domains:
        domains[domain] = 0
    domains[domain] += 1

print(f"Number of domains: {len(domains)}")
print(f"Number of domains with at least 4 pages: {sum([1 for (domain, x) in domains.items() if x >= 4])}")

Number of domains: 841520
Number of domains with at least 4 pages: 49076


#### Let's build a simple inverted index

In [42]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from unidecode import unidecode

# Download necessary data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/rossano/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rossano/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/rossano/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [47]:
def get_tokens(doc):   
    # Step 1: Convert document to lowercase
    doc = doc.lower()

    # Step 2: # Step 1: Normalize accents e.g., café vs cafe
    doc = unidecode(doc)
    
    # Step 3: Normalize multiple spaces to a single space
    doc = " ".join(doc.split())
    
    # Step 4: Tokenize the document
    tokens = word_tokenize(doc)
    
    # Step 5: Remove punctuation
    tokens_no_punct = [word for word in tokens if word not in string.punctuation]
    
    # Step 6: Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens_no_stopwords = [word for word in tokens_no_punct if word not in stop_words]
    
    # Step 7: Lemmatization (or stemming)
    #lemmatizer = WordNetLemmatizer()
    #tokens_lemmatized = [lemmatizer.lemmatize(word) for word in tokens_no_stopwords]
    
    # Step 8: Remove numbers
    tokens_no_numbers = [word for word in tokens_no_stopwords if not word.isdigit()]

    return tokens_no_numbers
    

In [48]:
get_tokens(documents[2])

['brighter',
 'communities',
 'worldwide',
 'undertakes',
 'many',
 'forms',
 'fundraising',
 'raise',
 'funds',
 'programmes',
 'kenya',
 'include',
 'organising',
 'events',
 'selling',
 'goods',
 'donations',
 'standing',
 'orders',
 'grant',
 'applications',
 'brighter',
 'communities',
 'worldwide',
 'committed',
 'achieving',
 'standards',
 'outlined',
 'statement',
 'guiding',
 'principles',
 'fundraising',
 'supplied',
 'charities',
 'institute',
 'ireland',
 'organisation',
 'representing',
 'interests',
 'irish',
 'charities',
 'good',
 'practice',
 'standards',
 'fundraising',
 'brighter',
 'communities',
 'worldwide',
 "'s",
 'accounts',
 'comply',
 'statement',
 'recommended',
 'practice',
 'sorp',
 'standard',
 'general',
 'dochas/irish',
 'aid',
 'guidelines',
 'financial',
 'reporting',
 'brighter',
 'communities',
 'worldwide',
 'publishes',
 'annual',
 'accounts',
 'line',
 'every',
 'year',
 'available',
 'website',
 'brighter',
 'communities',
 'worldwide',
 'regist

In [73]:
vocabulary_map = {}
frequencies = [] # for ith term, frequencies[i] stores its frequency
df = [] # for ith term, df[i] stores the number of documents containing this term
posting_lists = [] # list of lists. In real life, it is a big list with offsets. (Lists in Python are vectors!)
tfs = [] # list of lists. For each document, the frequency of the term in the document

for doc_id, document in enumerate(documents[:10]):
    tokens = get_tokens(document)
    
    for token in tokens: 
        if token not in vocabulary_map: # new term
            vocabulary_map[token] = len(vocabulary_map)
            posting_lists.append([])
            tfs.append([])
            frequencies.append(0)
            df.append(0)
            
        token_id = vocabulary_map[token]
        frequencies[token_id] += 1
        
        if len(posting_lists[token_id]) == 0 or posting_lists[token_id][-1] != doc_id: # avoid duplicated doc_ids within the same list
            posting_lists[token_id].append( doc_id )
            tfs[token_id].append( 0 )
            df[token_id] += 1
            
        tfs[token_id][-1] += 1



In [65]:
frequencies[:15]

[2, 1, 12, 1, 1, 1, 3, 2, 1, 1, 1, 15, 1, 2, 1]

In [66]:
sorted(vocabulary_map.items(), key = lambda x: x[1])[:15]

[('liposuction', 0),
 ('remained', 1),
 ('one', 2),
 ('popular', 3),
 ('cosmetic', 4),
 ('surgeries', 5),
 ('years', 6),
 ('people', 7),
 ('turn', 8),
 ('doctors', 9),
 ('remove', 10),
 ('fat', 11),
 ('diet', 12),
 ('exercise', 13),
 ('ca', 14)]

In [70]:
term = "one"

term_id = vocabulary_map[term]

list( zip(posting_lists[term_id], tfs[term_id]) )

[(0, 3), (1, 2), (3, 1), (4, 1), (5, 2), (6, 2), (9, 1)]

In [72]:
doc_id = posting_lists[term_id][0]
freq = tfs[term_id][0]

print( freq, documents[doc_id].count(term) )

3 3


-----

## Data Compression

#### Reorder documents sorting by URLs

**Goal**: Assign close *docIds* to documents from the same domain. **Why?**

Let's reverse the domain so that:

https://microsoft.github.io --> io.github.microsoft 

In [20]:
# https://microsoft.github.io --> io.github.microsoft
def get_domain_rev(domain):
    domain = domain.replace("https://", "").replace("http://", "")
    return ".".join(domain.split(".")[::-1])

def get_url_rev(url):
    url = url.replace("https://", "").replace("http://", "")
    split = url.split("/")
    return get_domain_rev(split[0]) + "/" + "/".join(split[1:])

In [16]:
print(get_domain_rev("https://microsoft.github.io"))

io.github.microsoft


In [18]:
print(get_url_rev("https://microsoft.github.io/presidio/getting_started/"))

io.github.microsoft/presidio/getting_started/


In [19]:
print(urls[0])
print(get_url_rev(urls[0]))

https://americanhealthandbeauty.com/articles/2704/non-surgical-fat-reduction--zerona-vs-zeltiq
com.americanhealthandbeauty/articles/2704/non-surgical-fat-reduction--zerona-vs-zeltiq


In [28]:
urls_rev = [get_url_rev(url) for url in urls]

print(urls_rev[0])

com.americanhealthandbeauty/articles/2704/non-surgical-fat-reduction--zerona-vs-zeltiq


In [24]:
permutation = list(range(len(urls)))

permutation.sort(key = lambda i: get_url_rev(urls[i]))

In [26]:
print(urls[permutation[0]])

https://www.accc.gov.au./media-release/accc-holiday-operations
